# Notebook Testing Fine-tuning with Unsloth

Reference Notebook: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install datasets
    !pip install -U bitsandbytes

In [ ]:
%load_ext autoreload
%autoreload 2

## Mount Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

FOLDERNAME = '493G1/Project/Adapt-main'

%cd drive/My\ Drive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/493G1/Project/Adapt-main


In [ ]:
from unsloth import FastLanguageModel
import torch
from src.Trainer import *
from datasets import load_dataset
import os

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


No NO_ASK_OPTION found in the environment variables.........
Setting to False as default.


No NO_UNDO_OPTION found in the environment variables.........
Setting to True as default.


No NO_CONSTRAINED_GENERATION found in the environment variables.........
Setting to False as default.

Trying to read /opt/hpcaas/.mounts/fs-03ee9f8c6dddfba21/jtruong/maithili/PolicyPersonalization/_sentence_embeddings_minilm.pt again...
Trying to read /opt/hpcaas/.mounts/fs-03ee9f8c6dddfba21/jtruong/maithili/PolicyPersonalization/_sentence_embeddings_minilm.pt again...
Trying to read /opt/hpcaas/.mounts/fs-03ee9f8c6dddfba21/jtruong/maithili/PolicyPersonalization/_sentence_embeddings_minilm.pt again...
Trying to read /opt/hpcaas/.mounts/fs-03ee9f8c6dddfba21/jtruong/maithili/PolicyPersonalization/_sentence_embeddings_minilm.pt again...
Trying to read /opt/hpcaas/.mounts/fs-03ee9f8c6dddfba21/jtruong/maithili/PolicyPersonalization/_sentence_embeddings_minilm.pt again...
Trying to read /opt/hpcaas/.mounts/fs

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# peft_model = FastLanguageModel.get_peft_model(
#     model,
#     r = 4, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0.01, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.01.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


## Try to call training functions

In [ ]:

OUT_FILENAME = "models/test"

trainable_model = LLMAgentTrainable(model, tokenizer, exact_outpath=OUT_FILENAME, weight_decay=0.001, learning_rate=1e-4, lora_rank=4, lora_alpha=16, lora_dropout=0.01, max_training_steps=500)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.01.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PEFT model loaded on devices:  {device(type='cuda', index=0)}


In [ ]:
data_filename = {split:os.path.join("data/split0",f"gen_dataset_DPO_unpriv_verbose_{split}_onlyAction.jsonl") for split in ['train', 'test']}
print(f"Using data from: \n  - {data_filename['train']}\n  - {data_filename['test']}")
datasets = {split:load_dataset("json", data_files=data_filename[split])["train"] for split in ['train', 'test']}
trainable_model.train_dpo(datasets, model)

Using data from: 
  - data/split0/gen_dataset_DPO_unpriv_verbose_train_onlyAction.jsonl
  - data/split0/gen_dataset_DPO_unpriv_verbose_test_onlyAction.jsonl
Training using DPO.....
Using dataset of length: 3169
------------------------------------------------------------------------------------------
Dataset Example:
{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are an expert at task planning, and know how to provide assistance in a manner that Jamal wants for preparing and serving breakfasts such as making cereal, pancakes, toast, waffles, french toast, coffee, tea, etc. You have the ability to take the following actions:\n- Open <X>: open an instance of an articulated furniture or object. e.g. Open cabinet\n- Close <X>: close an instance of an articulated furniture or object. e.g. Close cabinet\n- Heat <X>: heat a container, which is located on a heating appliance. e.g. Heat pan_0\n- Tu

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/3169 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/3169 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/3169 [00:00<?, ? examples/s]

Extracting prompt in eval dataset (num_proc=2):   0%|          | 0/353 [00:00<?, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hansellee (hansellee-university-of-washington) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
5,0.693100,0.000000,0.000000,0.000000,0.000000,-47.752106,-39.581581,-0.443236,-0.219696,0,0,0,0
10,0.693100,0.000000,0.000000,0.000000,0.000000,-50.431835,-42.207497,-0.445957,-0.364124,No Log,No Log,No Log,No Log
15,0.693100,0.000000,0.000000,0.000000,0.000000,-50.548378,-40.905556,-0.465513,-0.230906,No Log,No Log,No Log,No Log
20,0.693100,0.000000,0.000000,0.000000,0.000000,-42.959770,-44.735043,-0.332634,-0.273195,No Log,No Log,No Log,No Log
25,0.693100,0.000000,0.000000,0.000000,0.000000,-45.067997,-47.940388,-0.395805,-0.331612,No Log,No Log,No Log,No Log
30,0.693100,0.000000,0.000000,0.000000,0.000000,-44.549553,-46.979351,-0.367420,-0.223274,No Log,No Log,No Log,No Log
35,0.693100,0.000000,0.000000,0.000000,0.000000,-49.067524,-49.780060,-0.413848,-0.173316,No Log,No Log,No Log,No Log
40,0.693100,0.000000,0.000000,0.000000,0.000000,-48.280781,-56.772022,-0.580432,-0.384567,No Log,No Log,No Log,No Log


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
